In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Prueba SQL-Python").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 19:50:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df_vuelos = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("input/vuelos.csv")
df_pilotos = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("input/pilotos.csv")
df_vuelos=df_vuelos.withColumnRenamed("Aerolínea","Aerolinea") 


df_vuelos.createOrReplaceTempView("vw_vuelos")  # este es el nombre de la tabla
df_pilotos.createOrReplaceTempView("vw_pilotos")  # este es el nombre de la tabla

In [ ]:
# Se realiza la consulta usando directamente SQL como prueba
spark.sql("SELECT * FROM vw_vuelos").show()


+-------------+---------+------+-------+------------------+------+
|Codigo Piloto|Aerolinea|Origen|Destino|Minutos de retraso|OnTime|
+-------------+---------+------+-------+------------------+------+
|        43556|       10|   SAP|    HAJ|                40|  null|
|        43557|        5|   MIA|    MIA|                30|  null|
|        43558|        3|   FLL|    FLL|                20|  null|
|        43559|        1|   TEG|    SAP|                 0|  null|
|        43560|        1|   HAJ|    SAP|                50|  null|
|        43561|        8|   HHN|    SAP|                20|  null|
|        43562|        2|   SAP|    HAJ|               -30|  null|
|        43563|        8|   MIA|    HHN|               -30|  null|
|        43564|       10|   FLL|    SAP|               -30|  null|
|        43565|        7|   TEG|    TEG|                 0|  null|
|        43566|        4|   HAJ|    HAJ|               -70|  null|
|        43567|        6|   HHN|    HHN|               -40|  n

In [4]:
# Se realiza la consulta usando directamente SQL como prueba
spark.sql("SELECT * FROM vw_pilotos").show()


+-------------+---------------+
|Codigo Piloto|         Piloto|
+-------------+---------------+
|        43556|       John Max|
|        43557|   Jilles Vlank|
|        43558|    Jorge Mej�a|
|        43559|David Colindres|
|        43560|Maximilian Call|
|        43561|   Muilin Mills|
|        43562|    Gianni Falk|
|        43563|       Hous Hih|
|        43564|       Cho Ming|
|        43565|        Chao Ma|
|        43566|     Jos� Perez|
|        43567|    Davie Mixal|
|        43568|     Filg Gills|
|        43569|       Hung Cho|
|        43570|     Filen Filg|
|        43571| Arthur Maxwell|
|        43572| Carlos Miranda|
|        43573|  Denis Tercero|
|        43574|   David German|
|        43575|   Charles Moll|
+-------------+---------------+
only showing top 20 rows



# SQL
#### Para todas las consultas se puede agregar un limit 1, sin embargo, para poder contrastar un poco la data se decide traer todo.

#### ¿Qué aerolínea tiene más vuelos?

In [5]:
spark.sql("SELECT Aerolinea, count(1) FROM vw_vuelos GROUP BY 1 ORDER BY 2 DESC").show()

+---------+--------+
|Aerolinea|count(1)|
+---------+--------+
|        7|     457|
|        5|     448|
|        3|     443|
|        9|     435|
|        1|     431|
|        6|     430|
|        8|     420|
|        2|     417|
|       10|     413|
|        4|     408|
+---------+--------+



#### ¿Qué Origen se repite más?

In [6]:
spark.sql("SELECT Origen, count(1) FROM vw_vuelos GROUP BY 1 ORDER BY 2 DESC").show()

+------+--------+
|Origen|count(1)|
+------+--------+
|   SAP|    1404|
|   FLL|     681|
|   MIA|     649|
|   HAJ|     644|
|   TEG|     489|
|   HHN|     435|
+------+--------+



#### ¿Desde donde vuela más la aerolínea 8?

In [7]:
spark.sql("SELECT Origen, count(1) FROM vw_vuelos WHERE Aerolinea = 8 GROUP BY 1 ORDER BY 2 DESC").show()

+------+--------+
|Origen|count(1)|
+------+--------+
|   SAP|     134|
|   HAJ|      68|
|   FLL|      63|
|   MIA|      60|
|   TEG|      58|
|   HHN|      37|
+------+--------+



#### ¿Hacia dónde vuela más la aerolínea 4?

In [8]:
spark.sql("SELECT Destino, count(1) FROM vw_vuelos WHERE Aerolinea = 4 GROUP BY 1 ORDER BY 2 DESC").show()

+-------+--------+
|Destino|count(1)|
+-------+--------+
|    SAP|     141|
|    HAJ|      70|
|    MIA|      61|
|    FLL|      56|
|    HHN|      44|
|    TEG|      36|
+-------+--------+



#### ¿Qué piloto vuela más?

In [9]:
spark.sql("""
    SELECT p.Piloto, v.`Codigo Piloto`, count(1) 
    FROM vw_vuelos v
    LEFT JOIN vw_pilotos p ON p.`Codigo Piloto` = v.`Codigo Piloto`
    GROUP BY 1, 2 
    ORDER BY 3 DESC
    """).show()

+---------------+-------------+--------+
|         Piloto|Codigo Piloto|count(1)|
+---------------+-------------+--------+
|   Jonh Pierson|        43579|    1028|
|  Peter Mariner|        43580|     937|
|Homer Williamns|        43581|     918|
|     Joe Hulson|        43576|     505|
|   Marie Malful|        43578|     452|
|David Colindres|        43559|      51|
|    Jorge Mej�a|        43558|      51|
| Miriam Willson|        43577|      39|
|   Muilin Mills|        43561|      30|
|Maximilian Call|        43560|      30|
|    Gianni Falk|        43562|      30|
| Arthur Maxwell|        43571|      24|
| Carlos Miranda|        43572|      24|
|  Denis Tercero|        43573|      24|
|   Jilles Vlank|        43557|      23|
|       John Max|        43556|      23|
|     Filg Gills|        43568|      15|
|   Charles Moll|        43575|      12|
|        Chao Ma|        43565|      12|
|   David German|        43574|      12|
+---------------+-------------+--------+
only showing top

# Python

#### En primer instancia se deben cargar los archivos vuelos y pilotos a un dataframe

In [10]:
df_vuelos = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("vuelos.csv")
df_pilotos = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("pilotos.csv")


In [11]:
df_vuelos

DataFrame[Codigo Piloto: int, Aerolínea: int, Origen: string, Destino: string, Minutos de retraso: int, OnTime: string]

In [12]:
type(df_vuelos)

pyspark.sql.dataframe.DataFrame

In [13]:
type(df_pilotos)

pyspark.sql.dataframe.DataFrame

#### Agregar en la hoja Vuelos un campo para el nombre del piloto
#### Insertar el nombre del piloto

In [14]:
from pyspark.sql.functions import col

In [15]:
df_vuelos = df_vuelos.withColumnRenamed("Aerolínea","Aerolinea") 

In [16]:
df_vuelos = df_vuelos.withColumnRenamed("Codigo Piloto", "Codigo_Piloto")

In [17]:
df_vuelos

DataFrame[Codigo_Piloto: int, Aerolinea: int, Origen: string, Destino: string, Minutos de retraso: int, OnTime: string]

In [18]:
df_vuelos = df_vuelos.join(df_pilotos, col("Codigo_Piloto") == col("`Codigo Piloto`"), "left")


In [19]:
df_vuelos = df_vuelos.drop("Codigo Piloto")

In [20]:
df_vuelos.show(3)

+-------------+---------+------+-------+------------------+------+------------+
|Codigo_Piloto|Aerolinea|Origen|Destino|Minutos de retraso|OnTime|      Piloto|
+-------------+---------+------+-------+------------------+------+------------+
|        43556|       10|   SAP|    HAJ|                40|  null|    John Max|
|        43557|        5|   MIA|    MIA|                30|  null|Jilles Vlank|
|        43558|        3|   FLL|    FLL|                20|  null| Jorge Mej�a|
+-------------+---------+------+-------+------------------+------+------------+
only showing top 3 rows



In [21]:
df_vuelos.toPandas().to_excel("output/b y c.xlsx", index=False)

#### Descartar/marcar los registros donde Origen y Destino sean iguales

In [22]:
from pyspark.sql.functions import when

In [23]:
df_vuelos = df_vuelos.withColumn("Marca", when(df_vuelos.Origen == df_vuelos.Destino, "x").otherwise(""))


In [24]:
df_vuelos.show(3)

+-------------+---------+------+-------+------------------+------+------------+-----+
|Codigo_Piloto|Aerolinea|Origen|Destino|Minutos de retraso|OnTime|      Piloto|Marca|
+-------------+---------+------+-------+------------------+------+------------+-----+
|        43556|       10|   SAP|    HAJ|                40|  null|    John Max|     |
|        43557|        5|   MIA|    MIA|                30|  null|Jilles Vlank|    x|
|        43558|        3|   FLL|    FLL|                20|  null| Jorge Mej�a|    x|
+-------------+---------+------+-------+------------------+------+------------+-----+
only showing top 3 rows



In [25]:
df_vuelos.toPandas().to_excel("output/d.xlsx", index=False)

#### Agregar comentario en ONTIME, si el tiempo en valor absoluto es menor o igual a 30 A, si es esta entre 30 y 50 B, si es mayor que 50 C

In [26]:
from pyspark.sql.functions import abs

In [27]:
df_vuelos = df_vuelos.withColumnRenamed("Minutos de retraso", "Minutos_de_retraso")

In [28]:
df_vuelos = df_vuelos.withColumn("OnTime", 
                   when(abs(df_vuelos.Minutos_de_retraso) <= 30, 'A').
                   when((abs(df_vuelos.Minutos_de_retraso) > 30) & (abs(df_vuelos.Minutos_de_retraso) <= 50), 'B').
                   when(abs(df_vuelos.Minutos_de_retraso) > 50, 'C').otherwise(''))

df_vuelos.show()


+-------------+---------+------+-------+------------------+------+---------------+-----+
|Codigo_Piloto|Aerolinea|Origen|Destino|Minutos_de_retraso|OnTime|         Piloto|Marca|
+-------------+---------+------+-------+------------------+------+---------------+-----+
|        43556|       10|   SAP|    HAJ|                40|     B|       John Max|     |
|        43557|        5|   MIA|    MIA|                30|     A|   Jilles Vlank|    x|
|        43558|        3|   FLL|    FLL|                20|     A|    Jorge Mej�a|    x|
|        43559|        1|   TEG|    SAP|                 0|     A|David Colindres|     |
|        43560|        1|   HAJ|    SAP|                50|     B|Maximilian Call|     |
|        43561|        8|   HHN|    SAP|                20|     A|   Muilin Mills|     |
|        43562|        2|   SAP|    HAJ|               -30|     A|    Gianni Falk|     |
|        43563|        8|   MIA|    HHN|               -30|     A|       Hous Hih|     |
|        43564|      

In [29]:
df_vuelos.toPandas().to_excel("output/e.xlsx", index=False)

#### ¿Quién es el piloto que tiene más vuelos A?

In [30]:
from pyspark.sql.functions import count, desc

In [31]:
counts = df_vuelos.groupBy("Codigo_Piloto","Piloto").agg(count("*").alias("count"))

In [32]:
counts.orderBy(desc("count")).show()

+-------------+---------------+-----+
|Codigo_Piloto|         Piloto|count|
+-------------+---------------+-----+
|        43579|   Jonh Pierson| 1028|
|        43580|  Peter Mariner|  937|
|        43581|Homer Williamns|  918|
|        43576|     Joe Hulson|  505|
|        43578|   Marie Malful|  452|
|        43559|David Colindres|   51|
|        43558|    Jorge Mej�a|   51|
|        43577| Miriam Willson|   39|
|        43560|Maximilian Call|   30|
|        43561|   Muilin Mills|   30|
|        43562|    Gianni Falk|   30|
|        43573|  Denis Tercero|   24|
|        43572| Carlos Miranda|   24|
|        43571| Arthur Maxwell|   24|
|        43556|       John Max|   23|
|        43557|   Jilles Vlank|   23|
|        43568|     Filg Gills|   15|
|        43565|        Chao Ma|   12|
|        43574|   David German|   12|
|        43564|       Cho Ming|   12|
+-------------+---------------+-----+
only showing top 20 rows



In [33]:
counts.orderBy(desc("count")).first()

Row(Codigo_Piloto=43579, Piloto='Jonh Pierson', count=1028)

In [34]:
counts.toPandas().to_excel("output/f.xlsx", index=False)

#### ¿Qué aerolínea tiene más vuelos C?

In [35]:
df_vuelos_ontime_c = df_vuelos.filter("OnTime = 'C'")

In [36]:
df_vuelos_ontime_c = df_vuelos_ontime_c.groupBy("Aerolinea").agg(count("*").alias("count"))

In [37]:
df_vuelos_ontime_c.orderBy(desc("count")).show()

+---------+-----+
|Aerolinea|count|
+---------+-----+
|        7|   78|
|        4|   75|
|        6|   69|
|        5|   69|
|        9|   68|
|        1|   64|
|        3|   60|
|        8|   55|
|        2|   55|
|       10|   46|
+---------+-----+



In [38]:
df_vuelos_ontime_c.toPandas().to_excel("output/g.xlsx", index=False)

#### ¿Para qué aerolínea vuela Hung Cho?

In [39]:
df_vuelos_hung_cho = df_vuelos.filter("Piloto = 'Hung Cho'")

In [40]:
df_vuelos_hung_cho.show()

+-------------+---------+------+-------+------------------+------+--------+-----+
|Codigo_Piloto|Aerolinea|Origen|Destino|Minutos_de_retraso|OnTime|  Piloto|Marca|
+-------------+---------+------+-------+------------------+------+--------+-----+
|        43569|        9|   TEG|    MIA|                 0|     A|Hung Cho|     |
|        43569|        4|   TEG|    SAP|               -10|     A|Hung Cho|     |
|        43569|        9|   HHN|    TEG|                10|     A|Hung Cho|     |
|        43569|        8|   FLL|    TEG|               -20|     A|Hung Cho|     |
|        43569|        3|   FLL|    TEG|                10|     A|Hung Cho|     |
+-------------+---------+------+-------+------------------+------+--------+-----+



In [41]:
df_vuelos_hung_cho.select("Aerolinea").distinct().show()

+---------+
|Aerolinea|
+---------+
|        3|
|        9|
|        4|
|        8|
+---------+



In [42]:
df_vuelos_ontime_c.toPandas().to_excel("output/h.xlsx", index=False)

#### ¿Cuántos vuelos A, B, C tiene Chao Ma?

In [43]:
df_vuelos_chao_ma = df_vuelos.filter("Piloto = 'Chao Ma'")

In [44]:
df_vuelos_chao_ma.show(15)

+-------------+---------+------+-------+------------------+------+-------+-----+
|Codigo_Piloto|Aerolinea|Origen|Destino|Minutos_de_retraso|OnTime| Piloto|Marca|
+-------------+---------+------+-------+------------------+------+-------+-----+
|        43565|        7|   TEG|    TEG|                 0|     A|Chao Ma|    x|
|        43565|        1|   HHN|    SAP|               -10|     A|Chao Ma|     |
|        43565|        2|   HAJ|    HHN|               -20|     A|Chao Ma|     |
|        43565|        8|   HAJ|    MIA|                70|     C|Chao Ma|     |
|        43565|        3|   HHN|    HAJ|               -20|     A|Chao Ma|     |
|        43565|        4|   TEG|    FLL|               -20|     A|Chao Ma|     |
|        43565|        5|   HAJ|    FLL|                 0|     A|Chao Ma|     |
|        43565|        5|   MIA|    FLL|                40|     B|Chao Ma|     |
|        43565|       10|   MIA|    TEG|               -10|     A|Chao Ma|     |
|        43565|        2|   

In [45]:
df_vuelos_chao_ma.groupBy("OnTime").agg(count("*").alias("count")).show()

+------+-----+
|OnTime|count|
+------+-----+
|     B|    1|
|     C|    2|
|     A|    9|
+------+-----+



In [46]:
df_vuelos_ontime_c.toPandas().to_excel("output/i.xlsx", index=False)